<a href="https://colab.research.google.com/github/Pulkit-g03/Anomaly-Detection-/blob/main/Autoencoders.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import tensorflow as tf

from tensorflow.keras.models import Model
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.feature_extraction.text import CountVectorizer

mpl.rcParams['figure.figsize'] = (10, 5)
mpl.rcParams['axes.grid'] = False

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/IPD/HDFS_2k.log_structured.csv')
and_df =pd.read_csv('/content/drive/MyDrive/IPD/Android_2k.log_structured.csv')
df= pd.concat([df, and_df], ignore_index=True)
df

,LineId,Date,Time,Pid,Level,Component,Content,EventId,EventTemplate,Tid
0,1,81109,203615,148,INFO,dfs.DataNode$PacketResponder,PacketResponder 1 for block blk_38865049064139...,E10,PacketResponder <*> for block blk_<*> terminating,NaN
1,2,81109,203807,222,INFO,dfs.DataNode$PacketResponder,PacketResponder 0 for block blk_-6952295868487...,E10,PacketResponder <*> for block blk_<*> terminating,NaN
2,3,81109,204005,35,INFO,dfs.FSNamesystem,BLOCK* NameSystem.addStoredBlock: blockMap upd...,E6,BLOCK* NameSystem.addStoredBlock: blockMap upd...,NaN
3,4,81109,204015,308,INFO,dfs.DataNode$PacketResponder,PacketResponder 2 for block blk_82291938032499...,E10,PacketResponder <*> for block blk_<*> terminating,NaN
4,5,81109,204106,329,INFO,dfs.DataNode$PacketResponder,PacketResponder 2 for block blk_-6670958622368...,E10,PacketResponder <*> for block blk_<*> terminating,NaN
...,...,...,...,...,...,...,...,...,...,...
3995,1996,03-17,16:16:08.840,1702,I,DisplayPowerController,HBM brightnessOut =38,E65,HBM brightnessOut =<*>,1820.0
3996,1997,03-17,16:16:08.840,1702,D,DisplayPowerController,"Animating brightness: target=38, rate=200",E21,"Animating brightness: target=<*>, rate=<*>",1820.0
3997,1998,03-17,16:16:09.141,1702,I,DisplayPowerController,HBM brightnessIn =38,E64,HBM brightnessIn =<*>,1820.0
3998,1999,03-17,16:16:09.141,1702,I,DisplayPowerController,HBM brightnessOut =38,E65,HBM brightnessOut =<*>,1820.0


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4000 entries, 0 to 3999
Data columns (total 10 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   LineId         4000 non-null   int64  
 1   Date           4000 non-null   object 
 2   Time           4000 non-null   object 
 3   Pid            4000 non-null   int64  
 4   Level          4000 non-null   object 
 5   Component      4000 non-null   object 
 6   Content        4000 non-null   object 
 7   EventId        4000 non-null   object 
 8   EventTemplate  4000 non-null   object 
 9   Tid            2000 non-null   float64
dtypes: float64(1), int64(2), object(7)
memory usage: 312.6+ KB


In [ ]:
vectorizer = CountVectorizer(max_features=500)
log_vectors = vectorizer.fit_transform(df['Content']).toarray()

In [ ]:
numerical_features = df.select_dtypes(include=np.number).columns.tolist()
x_train, x_test, y_train, y_test = train_test_split(
    df[numerical_features].values,
    df.values[:, 0:1],
    test_size=0.2,
    random_state=111
)

scaler = MinMaxScaler()
data_scaled = scaler.fit(x_train)
train_data_scaled = data_scaled.transform(x_train)
test_data_scaled = data_scaled.transform(x_test)

In [ ]:
normal_data = df[df['Level'] == 'INFO']
anomaly_data = df[df['Level'].isin(['ERROR', 'WARN'])]

normal_train, normal_test = train_test_split(normal_data, test_size=0.2, random_state=42)

anomaly_train, anomaly_test = train_test_split(anomaly_data, test_size=0.2, random_state=42)

normal_train.to_csv('normal_train.csv', index=False)
normal_test.to_csv('normal_test.csv', index=False)
anomaly_train.to_csv('anomaly_train.csv', index=False)
anomaly_test.to_csv('anomaly_test.csv', index=False)

In [ ]:
model = tf.keras.Sequential()
model.add(tf.keras.layers.Dense(64, activation="relu"))
model.add(tf.keras.layers.Dense(32, activation="relu"))
model.add(tf.keras.layers.Dense(16, activation="relu"))
model.add(tf.keras.layers.Dense(8, activation="relu"))
model.add(tf.keras.layers.Dense(16, activation="relu"))
model.add(tf.keras.layers.Dense(32, activation="relu"))
model.add(tf.keras.layers.Dense(64, activation="relu"))
model.add(tf.keras.layers.Dense(140, activation="sigmoid"))

In [ ]:
class AutoEncoder(Model):
  def __init__(self):
    super(AutoEncoder, self).__init__()
    self.encoder = tf.keras.Sequential([
                  tf.keras.layers.Dense(64, activation="relu"),
                  tf.keras.layers.Dense(32, activation="relu"),
                  tf.keras.layers.Dense(16, activation="relu"),
                  tf.keras.layers.Dense(8, activation="relu")
              ])
    self.decoder = tf.keras.Sequential([
                  tf.keras.layers.Dense(16, activation="relu"),
                  tf.keras.layers.Dense(32, activation="relu"),
                  tf.keras.layers.Dense(64, activation="relu"),
                  tf.keras.layers.Dense(140, activation="sigmoid")
              ])
  def call(self, x):
    encoded = self.encoder(x)
    decoded = self.decoder(encoded)
    return decoded

In [ ]:
scaler = MinMaxScaler()
log_vectors_scaled = scaler.fit_transform(log_vectors)

train_data, test_data = train_test_split(log_vectors_scaled, test_size=0.2, random_state=42)

class AutoEncoder(tf.keras.Model):
    def __init__(self, input_dim):
        super(AutoEncoder, self).__init__()
        self.encoder = tf.keras.Sequential([
            tf.keras.layers.Dense(128, activation="relu"),
            tf.keras.layers.Dense(64, activation="relu"),
            tf.keras.layers.Dense(32, activation="relu"),
            tf.keras.layers.Dense(16, activation="relu")
        ])
        self.decoder = tf.keras.Sequential([
            tf.keras.layers.Dense(32, activation="relu"),
            tf.keras.layers.Dense(64, activation="relu"),
            tf.keras.layers.Dense(128, activation="relu"),
            tf.keras.layers.Dense(input_dim, activation="sigmoid")
        ])

    def call(self, x):
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)
        return decoded

input_dim = log_vectors_scaled.shape[1]
model = AutoEncoder(input_dim)
model.compile(optimizer='adam', loss='mae')

early_stopping = tf.keras.callbacks.EarlyStopping(monitor="val_loss", patience=3, mode="min")

history = model.fit(
    train_data, train_data,
    epochs=50,
    batch_size=32,
    validation_split=0.2,
    shuffle=True,
    callbacks=[early_stopping]
)

reconstructed_data = model.predict(test_data)
reconstruction_error = np.mean(np.abs(test_data - reconstructed_data), axis=1)

threshold = np.percentile(reconstruction_error, 80)
anomalies = reconstruction_error > threshold

print(f"Number of Anomalies Detected: {np.sum(anomalies)}")

Epoch 1/50
80/80 ━━━━━━━━━━━━━━━━━━━━ 5s 6ms/step - loss: 0.2953 - val_loss: 0.0143
Epoch 2/50
80/80 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.0146 - val_loss: 0.0143
Epoch 3/50
80/80 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 0.0147 - val_loss: 0.0144
Epoch 4/50
80/80 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 0.0146 - val_loss: 0.0142
Epoch 5/50
80/80 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.0144 - val_loss: 0.0136
Epoch 6/50
80/80 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 0.0136 - val_loss: 0.0134
Epoch 7/50
80/80 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 0.0139 - val_loss: 0.0134
Epoch 8/50
80/80 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 0.0137 - val_loss: 0.0134
Epoch 9/50
80/80 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step - loss: 0.0138 - val_loss: 0.0134
Epoch 10/50
80/80 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - loss: 0.0136 - val_loss: 0.0134
Epoch 11/50
80/80 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - loss: 0.0135 - val_loss: 0.0134
Epoch 12/50
80/80 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step - loss: 0.0134 - va